In [1]:
import json
import pandas as pd
import csv
import random
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats

# Utils

In [2]:
def get_ids(df):
    conversations = df.values.tolist()
    conversation_ids = []
    for conv in conversations:
        cur_conv = json.loads(conv)
        conversation_ids.append(list(cur_conv.keys())[0])
    return conversation_ids

In [3]:
def non_readable_or_not_sure(df):
    df['not_English'] = (df['na'] == True)
    df['not_sure_toxicity'] = (df['toxic'] == -1)
    non_english = pd.DataFrame({'not_readable': df.groupby('_unit_id')['not_English'].mean() >=0.5}).reset_index()
    non_english = non_english[non_english['not_readable'] == True]['_unit_id'].values.tolist()
    not_sure_toxicity = pd.DataFrame({'not_sure_toxicity': df.groupby('_unit_id')['not_sure_toxicity'].mean() >= 0.5}).reset_index()
    not_sure_toxicity = not_sure_toxicity[not_sure_toxicity['not_sure_toxicity'] == True]['_unit_id'].values.tolist()
    return non_english + not_sure_toxicity

# JOB 1

In [7]:
with open('/scratch/wiki_dumps/annotated/job1_constraintsAB_v017.csv') as f:
     df = pd.read_csv(f, encoding = 'utf-8', index_col=None, quoting=csv.QUOTE_ALL)

toxicity = df[df['toxic'] >= 0]
toxic_before = pd.DataFrame({'toxic_before': toxicity.groupby('_unit_id')['toxic'].mean() >= 0.5}).\
            reset_index()
toxic_before = toxic_before[toxic_before['toxic_before'] == True]['_unit_id'].values.tolist()
_excluded = list(set(non_readable_or_not_sure(df) + toxic_before))
annotated = set(get_ids(df['conversations']))
excluded_1 = set(get_ids(df[df['_unit_id'].isin(_excluded)]['conversations']))

In [ ]:
# Each Conversation is annotated by 5~22 workers JOB1

# JOB 2

In [11]:
with open('/scratch/wiki_dumps/annotated/job2_constraintsAB_v017.csv') as f:
     df = pd.read_csv(f, encoding = 'utf-8', index_col=None, quoting=csv.QUOTE_ALL)

_excluded = list(set(non_readable_or_not_sure(df)))
excluded_2 = set(get_ids(df[df['_unit_id'].isin(_excluded)]['conversations']))
excluded = (excluded_1 | excluded_2)

In [12]:
df['conv_id'] = df.apply(lambda x: list(json.loads(x['conversations']).keys())[0], axis=1)

In [13]:
toxicity = df[df['toxic'] >= 0]
toxic = pd.DataFrame({'toxic': toxicity.groupby('_unit_id')['toxic'].mean() >= 0.5}).reset_index()
bad_convs = toxic[toxic['toxic'] == True]['_unit_id'].values.tolist()
bad_conversations = set(get_ids(df[df['_unit_id'].isin(bad_convs)]['conversations']))

In [14]:
df['has_disagreement'] = ~(df['disagreement'] == None)
disagreement = pd.DataFrame({'disagreement': df.groupby('_unit_id')['has_disagreement'].mean() >= 0.5}).reset_index()
disagree_convs = disagreement[disagreement['disagreement'] == True]['_unit_id'].values.tolist()
disagree_conversations = set(get_ids(df[df['_unit_id'].isin(disagree_convs)]['conversations']))

In [15]:
bad_and_disagree = (bad_conversations & disagree_conversations - excluded)
good_and_disagree = (disagree_conversations - bad_conversations - excluded)

In [17]:
def attacker_in_conv(conv):
    actions = conv['action_feature']
    end_time = max([a['timestamp_in_sec'] for a in actions])
    attacker = None
    for a in actions:
        if a['timestamp_in_sec'] == end_time:
            attacker = a['user_text']
    for a in actions:
        if a['timestamp_in_sec'] < end_time and a['user_text'] == attacker:
            return True
    return False

In [26]:
goods = {}
bads = []
all_data = {}
included = {}
with open('/scratch/wiki_dumps/expr_with_matching/delta2_no_users_attacker_in_conv/data/all.json') as f:
    for line in f:
        conv_id, clss, conversation = json.loads(line)
        if conv_id in all_data:
            continue
        all_data[conv_id] = [conv_id, clss, conversation]
        if clss == 0 and (attacker_in_conv(all_data[conversation['action_feature'][0]['good_conversation_id']][2])):
            bads.append([conv_id, clss, conversation])
            goods[conv_id] = all_data[conversation['action_feature'][0]['good_conversation_id']]
cleaned_data = bads + list(goods.values())

In [30]:
len(cleaned_data)

5078

In [32]:
data = []
bad_disagree = []
good_disagree = []
xs = []
total_bad = 0
total_good = 0
for line in cleaned_data:
    conv_id, clss, conversation = line
    idlist = [k['id'] for k in conversation['action_feature']]
    is_annotated = not(annotated.isdisjoint(idlist))
    is_excluded = not(excluded.isdisjoint(idlist))
    is_bd = not(bad_and_disagree.isdisjoint(idlist))
    is_gd = not(good_and_disagree.isdisjoint(idlist))
    annotated_class = bad_conversations.isdisjoint(idlist)
    same_class = (int(annotated_class) == clss)
    if 'good_conversation_id' in conversation['action_feature'][0]:
        matched_id = conversation['action_feature'][0]['good_conversation_id']
    else:
        matched_id = conversation['action_feature'][0]['bad_conversation_id']
    dt = {'conversation_id': conv_id, 'annotated': is_annotated, 'excluded': is_excluded, \
        'annotated_class': annotated_class, 'class': clss, 'matched_id': matched_id, 'same_class': same_class}
    if is_bd and not(is_excluded) and is_annotated:
        bad_disagree.append(dt)
    if is_gd and not(is_excluded):
        good_disagree.append(dt)
    if annotated_class:
        total_good += 1
    else:total_bad += 1
    data.append(dt)

In [40]:
bad_but_disagree = pd.DataFrame(bad_disagree)
good_but_disagree = pd.DataFrame(good_disagree)
print('Number of bad conversations with disagreement :', len(bad_but_disagree))
print('Total Number of bad conversations:', total_bad)
print('Number of good conversations with disagreement :', len(good_but_disagree))
print('Total Number of bad conversations:', total_good)
print('Good conversations with disagreement / good conversations:', len(good_but_disagree) / total_good)
print('Bad conversations with disagreement / bad conversations:', len(bad_but_disagree) / total_bad)
print('Bad conversations with disagreement / all conversations with disagreement', len(bad_but_disagree) / (len(good_but_disagree) + len(bad_but_disagree)))
print('Number of bad conversations / all conversations:', total_bad/(total_bad+total_good))

Number of bad conversations with disagreement : 1291
Total Number of bad conversations: 1334
Number of good conversations with disagreement : 2897
Total Number of bad conversations: 3744
Good conversations with disagreement / good conversations: 0.7737713675213675
Bad conversations with disagreement / bad conversations: 0.9677661169415293
Bad conversations with disagreement / all conversations with disagreement 0.30826170009551096
Number of bad conversations / all conversations: 0.26270185112248917


3602
4744
0.7592748735244519


In [152]:
1668/(1668+3602) # toxic&disagree / all disagree

0.31650853889943076

In [153]:
1720/(1720+4744) # toxic / all_data

0.2660891089108911

In [38]:
annotation_aggregated = pd.DataFrame(data)
print('Number of Conversations Annotated:', len(annotation_aggregated[annotation_aggregated['annotated'] == True]))
print('Number of Conversations Excluded:', len(annotation_aggregated[annotation_aggregated['excluded'] == True]))
annotation_aggregated = annotation_aggregated[(annotation_aggregated['annotated'] == True)\
                                              & (annotation_aggregated['excluded'] == False)\
                                              & (annotation_aggregated['same_class'] == True)]
left_ids = annotation_aggregated['conversation_id'].values.tolist()
annotation_aggregated = annotation_aggregated[annotation_aggregated['matched_id'].isin(left_ids)]
print('Number of Conversations left:', len(annotation_aggregated))

Number of Conversations Annotated: 4306
Number of Conversations Excluded: 118
Number of Conversations left: 2066


In [15]:
verified_pairs = annotation_aggregated['conversation_id'].values.tolist()
goods = {}
bads = []
all_data = {}
with open('/scratch/wiki_dumps/expr_with_matching/delta2_no_users_attacker_in_conv/data/all_verified.json', 'w') as w:
    with open('/scratch/wiki_dumps/expr_with_matching/delta2_no_users_attacker_in_conv/data/all.json') as f:
        for line in f:
            conv_id, clss, conversation = json.loads(line)
            if conv_id in verified_pairs:
                w.write(json.dumps([conv_id, clss, conversation]) + '\n')
                all_data[conv_id] = [conv_id, clss, conversation]
                if clss == 0:
                    bads.append([conv_id, clss, conversation])
                    goods[conv_id] = all_data[conversation['action_feature'][0]['good_conversation_id']]

# Annotator Agreement

In [9]:
TOXIC_COLUMNS = ['no_toxic', 'toxic']

no_toxic = pd.DataFrame({'no_toxic': df[df['toxic'] == 0].groupby('_unit_id').size()}).reset_index().set_index('_unit_id')
toxic = pd.DataFrame({'toxic': df[df['toxic'] == 1].groupby('_unit_id').size()}).reset_index().set_index('_unit_id')
total = toxic.join(no_toxic, how='outer')
total = total.fillna(int(0)).reset_index()
total['sum'] = total['toxic'] + total['no_toxic']
#total = total[total['sum'] ==20]

len(total[total['sum'] >=20]) / len(total)

total = total[total['sum'] == 5]

total['toxic'] = total.apply(lambda x: int(x['toxic']), axis=1)

total['sum'] = total['toxic'] + total['no_toxic']

total[['sum']].plot.hist

n,bins,batches = plt.hist(total['no_toxic'])

len(total[total['no_toxic'] > 20]) / len(total)

plt.show()

total = total[~(total['_unit_id'].isin(_excluded))]

print(len(total), Krippendorf_alpha(total, TOXIC_COLUMNS))

In [164]:
TOXIC_COLUMNS = ['no_toxic', 'toxic']

no_toxic = pd.DataFrame({'no_toxic': df[df['toxic'] == 0].groupby('_unit_id').size()}).reset_index().set_index('_unit_id')
toxic = pd.DataFrame({'toxic': df[df['toxic'] == 1].groupby('_unit_id').size()}).reset_index().set_index('_unit_id')
total = toxic.join(no_toxic, how='outer')
total = total.fillna(0).reset_index()
total['sum'] = total['toxic'] + total['no_toxic']
total = total[total['sum'] ==20]

2643 0.282819056548
